In [ ]:
#!/usr/bin/env python
# coding: utf-8

# In[7]:


import numpy as np
from gurobipy import *
import os
import time


# In[2]:


from data_generation_func import datageneration


# In[3]:


# S = [[0, 4, 5], [0, 0, 1], [0, 0, 0]]
# E = [5, 10, 8]
# T = [8, 15, 12]
# L = [10, 20, 15]
# g = [10, 5, 5]
# h = [5, 5, 10]
# n = 3

#### generating instances here
# n = [3, 5, 10]
# instace = 10

# for i in n:
#     for j in range(instace):
#         datageneration(i,j)


# In[4]:


def mip(n, S, T, E, L, g, h):
    ## from data generation file - num of planes, separation time, earliest, latest, target, penalty for early, penalty for late
#     n, S, E, L, T, g, h = # some function 
    ### figure out the set of U, V and W
    m = Model()
    m.setParam("LogToConsole", 0)
    
    x = m.addVars(n, vtype = GRB.INTEGER) ## arrival time of each plane
    U = []
    V = []
    W = []
    ## getting pairs into U, V, W
    for i in range(n):
        for j in range(n):
            if i != j:
                if L[i] < E[j] and L[i] + S[i][j] <= E[j]:
                    W.append((i,j))
                elif L[i] < E[j] and L[i] + S[i][j] > E[j]:
                    V.append((i,j))
                else:
                    U.append((i,j))
    ## delta
    delta = m.addVars(n,n, vtype = GRB.BINARY)
    for i in range(n): 
        for j in range(n):
            if j > i:
                m.addConstr(delta[i,j] + delta[j,i] == 1) ## constraint 6  
            if i != j:
                if (i,j) in W+V:
                    m.addConstr(delta[i, j] == 1)
                if (i,j) in U:
                    m.addConstr(x[j] - x[i] >= S[i][j]*delta[i,j] - (L[i] - E[i])*delta[j, i])
    
    alpha = m.addVars(n, lb = 0, vtype = GRB.INTEGER) # arriving early
    beta = m.addVars(n, lb = 0, vtype = GRB.INTEGER) # arriving late
    
    for i in range(n): 
        m.addConstr(x[i] <= L[i])
        m.addConstr(x[i] >= E[i])
        m.addConstr(alpha[i] >= T[i] - x[i])
#         m.addConstr(alpha[i] >= 0)
        m.addConstr(alpha[i] <= T[i] - E[i])
        
        m.addConstr(beta[i] >= x[i] - T[i])
#         m.addConstr(beta[i] >= 0)
        m.addConstr(beta[i] <= L[i] - T[i])
        
        m.addConstr(x[i] == T[i] - alpha[i] + beta[i])
        
    m.update() 
    m.setObjective(quicksum(g[i]*alpha[i] + h[i]*beta[i] for i in range(n)), GRB.MINIMIZE) ## min total weighted completion time 
    m.update()
#     m.optimize()
    
    s1 = time.time()
    m.optimize()
    solve_time = time.time() - s1
    
    optimal_x = [x[i].X for i in range(n)]
    optimal_cost = m.objVal
    
    return n, optimal_x, optimal_cost, m.NumConstrs, solve_time


# In[5]:


# T[i], E[i], L[i], G[i], H[i])
def read_instance(instance):
    T = []
    E = []
    L = []
    g = []
    h = []
    S = []
    with open(instance) as f:
        for line in f:
            if 's_ij' in line:
                break ### a rather silly way to do this but cant figure out a better way atm perhaps we 
                ## need a different format for the instances 
            param = [int(i) for i in line.split()]
            T.append(param[0])
            E.append(param[1])
            L.append(param[2])
            g.append(param[3])
            h.append(param[4])
        for line in f:
            S.append([int(i) for i in line.split()])
    n = len(T)
    return n, S, T, E, L, g, h


# In[8]:


# n, S, T, E, L, g, h = read_instance('Instances/3planes1runway-4.txt')
# n, optimal_x, optimal_cost, num_constr, solve_time = mip(n, S, T, E, L, g, h)


# ## Output logs

# In[11]:


### read and output 
f = open('MIP_output.csv', 'a')
f.write('Model, n, Instance, NumConstr, Opt_Cost, Solve_time' + '\n')
for file in os.listdir("Instances/"):
    if file.startswith('3'):
        idx = file[-5]
        n, S, T, E, L, g, h = read_instance('Instances/'+file)
        try: ### trying to solve the instance if unsolvable continue 
            n, optimal_x, optimal_cost, num_constr, solve_time = mip(n, S, T, E, L, g, h)
            f.write('MIP' + ',' + str(n) + ',' + str(idx) + ','
                    + str(num_constr) + ',' + str(optimal_cost) + ',' + str(solve_time)
                    + '\n'
                    )
            #### for cp 
            ## call it here 
            ## f.write('CP' + other things)
        except: continue
f.close()


# In[ ]:



